# Creating a synthetic Q&A dataset

In [15]:
import os

proxy = 'http://127.0.0.1:7890' # clash
# proxy = 'http://127.0.0.1:1080' # naiveproxy

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

## Read in data, and create a context

In [24]:
import pandas as pd
df = pd.read_csv("docx/total.csv")
df['context'] = df.summary + "\n" + df.content
df.head()

,summary,content,summary_token_num,content_token_num,token_num,context
0,基础知识#业务介绍#网络的定义,是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能够提供包...,8,45,53,基础知识#业务介绍#网络的定义\n是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结...
1,基础知识#业务介绍#联通优势,与网络相比：速度更快、选择更个性化、网络覆盖更宽广、业务更丰富。与其它标准相比：全球开通网络...,8,105,113,基础知识#业务介绍#联通优势\n与网络相比：速度更快、选择更个性化、网络覆盖更宽广、业务更丰...
2,基础知识#业务介绍#用户的定义,开通使用中国联通标准资费、差不多套餐、无线上网卡标准资费或套餐的用户。,8,19,27,基础知识#业务介绍#用户的定义\n开通使用中国联通标准资费、差不多套餐、无线上网卡标准资费或...
3,基础知识#业务介绍#后付费用户定义,后付费用户须签约和登记用户资料；先使用，后付费，按月缴费，享受客户俱乐部、积分等客户服务。可...,10,40,50,基础知识#业务介绍#后付费用户定义\n后付费用户须签约和登记用户资料；先使用，后付费，按月缴...
4,基础知识#业务介绍#标准产品〔预付费〕定义,标准产品〔预付费〕是实时扣费、先付费后使用的产品。,13,18,31,基础知识#业务介绍#标准产品〔预付费〕定义\n标准产品〔预付费〕是实时扣费、先付费后使用的产品。


## Create questions based on the context

In [25]:
import openai
openai.api_key_path = ".openai-key"

In [26]:

def get_questions(context):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"请根据以下文本生成问题\n\n文本: {context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response['choices'][0]['text']
    except:
        return ""

df['questions']= df.context.apply(get_questions)
df['questions'] = "1." + df.questions

In [28]:
print(df[['questions']].values)

[['1. 中国联通采用的是什么标准?\n2. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能够提供包括可视   、无线上网，手机上网、手机电视、手机音乐等多种信息服务。\n']
 ['1. 联通的优势与网络相比是什么?\n2. 与其它标准相比, 全球开通网络最多是什么?\n3. 全球市场份额最高是什么?\n4. 终端款式最丰富是什么?\n5. 商用时刻最长是什么?']
 ['1. 开通使用中国联通标准资费的用户是什么?\n2. 差不多套餐的用户是什么?\n3. 无线上网卡标准资费或套餐的用户是什么?']
 ['1.后付费用户是指什么?\n2.后付费用户必须签约和登记用户资料?\n3.后付费用户可以享受客户俱乐部、积分等客户服务?\n4.后付费用户可以对信用额度进行操纵?']
 ['1. 什么是标准产品〔预付费〕?\n2. 先付费后使用的产品是什么?\n3. 实时扣费是什么?']
 ['1. 使用标准产品〔预付费〕的用户，无须登记客户资料，先付费后使用，实时扣费；以自助服务为主，是否意味着用户无需提供任何个人信息?']
 ['1. 什么是无线上网卡?\n2. 什么是手机电视?\n3. 什么是手机音乐?\n4. 什么是手机上网?\n5. 什么是手机报?\n6. 什么是手机搜索?\n7. 什么是即时通信?']
 ['1. 城市开始试商用的日期是什么时候?\n2. 其他城市开始试商用的日期是什么时候?\n3. 个城市同步开始正式商用的日期是什么时候?\n4. 规模放号的日期是什么时候?']
 ['1. 月日至月日试商用期中，推出哪些产品?\n2. 上网卡用户推出的产品是基于北京系统的预付费标准产品，后续视业务进展情形适时推出基于各省的后付费产品。这意味着哪些用户可以享受到这些产品?\n3. 月日开始，推出一般用户〔不含上网卡用户〕标准产品〔预付费〕。']
 ['1. 业务介绍中提到的"月日之前"是指哪一天?\n2. 用户于月日早：点到月日晚：之前登陆中国联通网上营业厅〔〕进行预约，预约的内容包括：选择号码、套餐、办理的营业厅，并留下姓名、有效证件号与x。']
 ['1. 业务资费及特点是什么?\n2. 多媒体计价单位是什么?\n3. 文本计价单位是什么?\n4. 手机音乐、手机电视、手机上网内网、手机搜索业务按次以〝〔〕＋￥

## Create answers based on the context

In [29]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"根据以下文本生成问题的答案\n\n文本: {row.context}\n\n问题:\n{row.questions}\n\n答案:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""


df['answers']= df.apply(get_answers, axis=1)
df['answers'] = "1." + df.answers

This model's maximum context length is 2049 tokens, however you requested 4693 tokens (4436 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2258 tokens (2001 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2074 tokens (1817 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2275 tokens (2018 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 3070 tokens (2813 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2239 tokens (198

In [30]:
print(df[['answers']].values)

[['1. 中国联通采用的是通信标准。\n2. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能够提供包括可视   、无线上网，手机上网、手机电视、手机音乐等多种信息服务。']
 ['1. 联通的优势与网络相比是速度更快、选择更个性化、网络覆盖更宽广、业务更丰富。\n2. 与其它标准相比, 全球开通网络最多是占全球商用网络的。\n3. 全球市场份额最高是截至年底, 全球共有用户亿, 全球市场占有率。\n4. 终端款式最丰富是截至年底, 支持商用终端的款式数']
 ['1. 开通使用中国联通标准资费的用户是什么?\n开通使用中国联通标准资费的用户是那些使用中国联通标准资费的用户。\n\n2. 差不多套餐的用户是什么?\n差不多套餐的用户是那些使用差不多套餐的用户。\n\n3. 无线上网卡标准资费或套餐的用户是什么?\n无线上网卡标准资费或套餐的用户是那些使用无']
 ['1.后付费用户是指先使用后付费的用户。\n2.后付费用户必须签约和登记用户资料。\n3.后付费用户可以享受客户俱乐部、积分等客户服务。\n4.后付费用户可以对信用额度进行操纵。']
 ['1. 标准产品〔预付费〕是实时扣费、先付费后使用的产品。\n2. 先付费后使用的产品是标准产品〔预付费〕。\n3. 实时扣费是实时扣费、先付费后使用的产品。']
 ['1. 使用标准产品〔预付费〕的用户，无须登记客户资料，先付费后使用，实时扣费；以自助服务为主，意味着用户无需提供任何个人信息。']
 ['1. 无线上网卡是通过移动通信网络接入互联网获得音乐、资讯、娱乐、商务、生活等的信息服务业务。\n2. 手机电视是通过手机电视的手机客户端或门户〔、〕，利用流媒体、视频等技术在移动终端上观看视频节目的业务。\n3. 手机音乐是通过音乐手机客户端或音乐门户〔']
 ['1. 月日至月日为试商用期,第一批都市开始试商用限量放号，月日开始试商用规模放号；其他都市在月底前分批开始试商用限量放号；\n\n2. 月底前其他城市开始试商用限量放号；\n\n3. 个城市同步开始正式商用的日期是月日；\n\n4. 规模放号的日期是月日。']
 ['1. 月日至月日试商用期中，推出哪些产品?\n一般用户仅推出后付费用户，用户仅可选择差不多套餐；上网卡用户推出基于北京系统的预付费标准产品，后续

In [31]:
df = df.dropna().reset_index().drop('index',axis=1)
df.to_csv("docx/qa.csv", index=False)

## Debug proxy 

In [16]:
test_context = df.context[0]
response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"请根据以下文本生成问题\n\n文本: {test_context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=800,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )


In [17]:
print(response['choices'][0]['text'])

 中国联通采用的是什么标准?
2. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能够提供包括可视   、无线上网，手机上网、手机电视、手机音乐等多种信息服务。

